In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
from tqdm import tqdm
import shutil,os

os.mkdir("./Train")
os.mkdir("./Valid")

val_csv = pd.read_csv("../input/annotations-v2/val_info_v2.csv")

src_path = "../input/ifood-2019-fgvc6/val_set/val_set/"
dst_path = "./Valid/"
for i in tqdm(range(len(val_csv))):
    img = val_csv["ImageID"][i]
    cls = val_csv["ClassName"][i]
    if(os.path.exists(dst_path+cls)):
        _=shutil.copy(src_path+img,dst_path+cls)
    else:
        os.mkdir(dst_path+cls)
        _=shutil.copy(src_path+img,dst_path+cls)

train_csv = pd.read_csv("../input/annotations-v2/train_info_v2.csv")

src_path = "../input/ifood-2019-fgvc6/train_set/train_set/"
dst_path = "./Train/"
for i in tqdm(range(len(train_csv))):
    img = train_csv["ImageID"][i]
    cls = train_csv["ClassName"][i]
    if(os.path.exists(dst_path+cls)):
        _=shutil.copy(src_path+img,dst_path+cls)
    else:
        os.mkdir(dst_path+cls)
        _=shutil.copy(src_path+img,dst_path+cls)

In [ ]:
!pip install pretrainedmodels
import os,time
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import pretrainedmodels
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
data_dir = "./"
train_dir = os.path.join(data_dir,"Train/")
valid_dir = os.path.join(data_dir,"Valid/")



data_transform = {'train' : transforms.Compose([transforms.CenterCrop(331),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])]),

                   'valid': transforms.Compose([transforms.Resize(size=(331,331)),
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])])
    
}

train_data = datasets.ImageFolder(train_dir, transform=data_transform['train'])
valid_data = datasets.ImageFolder(valid_dir, transform=data_transform['valid'])

batch_size = 16
num_workers = 0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,num_workers=num_workers, shuffle=True)
data_loaders = {'train' : train_loader, 'valid' : valid_loader}

In [ ]:
#Checking if GPU is available
use_cuda = torch.cuda.is_available()
print("GPU available: ",use_cuda)

class FCWithLogSigmoid(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(FCWithLogSigmoid, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
        self.logsigmoid = nn.LogSigmoid()
    def forward(self, x):
        return self.logsigmoid(self.linear(x))

#Specifying model
model_name = 'polynet'
model = pretrainedmodels.__dict__[model_name](pretrained=None) #pretrained='imagenet+background'



for param in model.parameters():
    param.requires_grad = False

model.last_linear = FCWithLogSigmoid(2048, 251)

# for param in model.last_linear.parameters():
#     param.requires_grad=True
    
# for param in model.stage_b.parameters():
#     param.requires_grad=True
    
# for param in model.reduction_b.parameters():
#     param.requires_grad=True
    
# for param in model.stage_c.parameters():
#     param.requires_grad=True
    
model.load_state_dict(torch.load('../input/models-aug/pytorch_polynet.pt'))

if use_cuda:
    model = model.cuda()

In [ ]:
model

In [ ]:
!nvidia-smi

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss().cuda()
optimizer = optim.Adam(model.parameters(),amsgrad=True,lr=1e-4,betas=(0.9, 0.999),
                       eps=1e-8,weight_decay=0.0) 
scheduler = lr_scheduler.MultiStepLR(optimizer,milestones=[2])

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    for epoch in range(1, n_epochs+1):
        start = time.time()
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        correct = 0.
        total = 0.

        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in tqdm(enumerate(loaders['train'])):
            # move to GPU
            target=target.numpy()
            tar = np.zeros((target.size, 251), dtype=int)
            tar[np.arange(target.size), target] = 1
            target=torch.from_numpy(tar)
            target= target.float()
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))

        ######################    
        # validate the model #
        ######################
        with torch.no_grad():
            
            model.eval()
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                # move to GPU
                target=target.numpy()
                tar = np.zeros((target.size, 251), dtype=int)
                tar[np.arange(target.size), target] = 1
                target=torch.from_numpy(tar)
                target= target.float()
                
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                ## update the average validation loss
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(data)
                # calculate the batch loss
                loss = criterion(output, target)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
                # convert output probabilities to predicted class
                pred = output.data.max(1, keepdim=True)[1]
                # compare predictions to true label
                target=target.cpu().numpy()
                target = [np.where(r==1)[0][0] for r in target]
                pred = pred.cpu().numpy()
                for i in range(len(target)):
                    if(target[i]==pred[i]):
                        correct+=1
                        total+=1
                    else:
                        total+=1
                

        accuracy = (100. * correct / total)
        end = time.time()
        epoch_time = (end-start)/60
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \
              \tValidation Accuracy: {:.4f} \tTime taken in min : {:.4f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            accuracy,
            epoch_time
            ))

        ## save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
        scheduler.step(valid_loss)
            
    # return trained model
    return model

In [ ]:
# train the model
n_epochs = 3
model = train(n_epochs, data_loaders, model, optimizer, criterion, use_cuda, 'pytorch_polynet.pt')

In [ ]:
from IPython.display import FileLink
FileLink('pytorch_polynet.pt')

In [ ]:
#Making predictions on test data by making batches manually
from PIL import Image
test_path = "../input/ifood-2019-fgvc6/test_set/test_set/"
batch_size = 64
test_images = os.listdir(test_path)
images_complete = test_images[:(len(os.listdir(test_path))//batch_size)*batch_size]
images_partial = test_images[(len(os.listdir(test_path))//batch_size)*batch_size:]

transform = transforms.Compose([transforms.CenterCrop(331),
                                 transforms.ToTensor(), 
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])
  

data = []
image_names = []
for i in tqdm(range(len(images_complete))):
    image = Image.open(test_path+images_complete[i])
    image = transform(image).float()
    image = image.unsqueeze(0)
    data.append(image)
    image_names.append(images_complete[i])

    if (len(data) == batch_size):
        t = 0
        batch_index = []
        first_tensor_data = data[t]
        while(t<(batch_size-1)):
            second_tensor_data = data[t+1]
            data_tensor = torch.cat((first_tensor_data,second_tensor_data),dim=0)
            first_tensor_data = data_tensor
            t+=1
        if use_cuda:
            data_tensor = data_tensor.cuda()
        output = model(data_tensor)
        index = output.data.cpu().numpy()
        for k in index:
            mini_batch_index = k.argsort()[::-1][:3]
            batch_index.append(mini_batch_index)
            
        assert len(image_names) == len(batch_index)
        
        with open("pytorch_polynet.txt","a") as f:
            for l in range(len(image_names)):
                f.write(image_names[l]+','+str(batch_index[l][0])+' '+str(batch_index[l][1])+' '+
                        str(batch_index[l][2])+'\n')
        data = []
        image_names = []

data = []
image_names = []
for i in tqdm(range(len(images_partial))):
    image = Image.open(test_path+images_partial[i])
    image = transform(image).float()
    image = image.unsqueeze(0)
    data.append(image)
    image_names.append(images_partial[i])

    if (len(data) == len(images_partial)):
        t = 0
        batch_index = []
        first_tensor_data = data[t]
        while(t<(len(images_partial)-1)):
            second_tensor_data = data[t+1]
            data_tensor = torch.cat((first_tensor_data,second_tensor_data),dim=0)
            first_tensor_data = data_tensor
            t+=1
        if use_cuda:
            data_tensor = data_tensor.cuda()
        output = model(data_tensor)
        index = output.data.cpu().numpy()
        for k in index:
            mini_batch_index = k.argsort()[::-1][:3]
            batch_index.append(mini_batch_index)
            
        assert len(image_names) == len(batch_index)
        
        with open("pytorch_polynet.txt","a") as f:
            for l in range(len(image_names)):
                f.write(image_names[l]+','+str(batch_index[l][0])+' '+str(batch_index[l][1])+' '+
                        str(batch_index[l][2])+'\n')
        data = []
        image_names = []

In [ ]:
from IPython.display import FileLink
FileLink('pytorch_polynet.txt')